In [89]:
import numpy as np
from random import expovariate as exp

In [90]:
class PriorityQueue:
    def __init__(self, lambda1, lambda2, t1, t2):
        # Parameters
        self.lambda1 = lambda1
        self.lambda2 = lambda2
        self.t1 = t1
        self.t2 = t2
        # Initialize State Variables
        self.n1 = 0
        self.n2 = 0
        self.s = 0
        # Initialize Clock Variables
        self.ArrRes1 = exp(self.lambda1)
        self.ArrRes2 = exp(self.lambda2)
        self.ServRes = 0
        # Initialize Analytic Variables
        self.totalDelay1 = 0
        self.totalDelay2 = 0
        self.nbrServed1 = 0
        self.nbrServed2 = 0

    def updateServedNbr(self):
        if self.s == 1:
            self.nbrServed1 += 1
        elif self.s == 2:
            self.nbrServed2 += 1

    def updateDelay(self, deltaTime):
        self.totalDelay1 += self.n1*deltaTime # Update delay of 1
        self.totalDelay2 += self.n2*deltaTime # Update delay of 2

    def singleTransition(self):
        deltaTime = 0
        if self.s == 0: # No customers at service or in the queues
            if self.ArrRes1 < self.ArrRes2: 
                # Level 1 customer arrives first & get served
                # Gone through Ares1 time
                deltaTime = self.ArrRes1 
                self.ArrRes2 -= deltaTime
                self.ArrRes1 = exp(self.lambda1)
                self.ServRes = self.t1
                # State changes
                self.s = 1 
            else: 
                # Level 2 customer arrives first & get served
                # Gone through Ares2 time
                deltaTime = self.ArrRes2
                self.ArrRes1 -= deltaTime
                self.ArrRes2 = exp(self.lambda2)
                self.ServRes = self.t2
                # State changes
                self.s = 2
        else: # There is customers at service, may or may not have people in the queues
            if self.ServRes > min(self.ArrRes1, self.ArrRes2):
                # An arrival occurs before service completion, service state doesn't change
                if self.ArrRes1 < self.ArrRes2: # Level 1 customer comes first
                    # Gone through Ares1 time
                    deltaTime = self.ArrRes1
                    self.ArrRes2 -= deltaTime
                    self.ServRes -=  deltaTime
                    self.updateDelay(deltaTime)
                    self.ArrRes1 = exp(self.lambda1)
                    # State changes 
                    self.n1 += 1
                else: # Level 2 customer comes first
                    # Gone through Ares2 time
                    deltaTime = self.ArrRes2
                    self.ArrRes1 -= deltaTime
                    self.ServRes -= deltaTime
                    self.updateDelay(deltaTime)
                    self.ArrRes2 = exp(self.lambda2)
                    # State changes
                    self.n2 += 1
            else: 
                # Service completed before any arrival, service state changes
                if self.n1 > 0: # There is level1 customer waiting
                    # Gone through Sres time
                    deltaTime = self.ServRes
                    self.ArrRes1 -= deltaTime
                    self.ArrRes2 -= deltaTime
                    self.ServRes = self.t1
                    self.updateDelay(deltaTime)
                    # State changes
                    self.n1 -= 1
                    self.updateServedNbr()
                    self.s = 1
                else: # No Level1 customer
                    if self.n2 > 0: # There is level2 customer waiting
                        deltaTime = self.ServRes
                        self.ArrRes1 -= deltaTime
                        self.ArrRes2 -= deltaTime
                        self.ServRes = self.t2
                        self.updateDelay(deltaTime)
                        # State changes
                        self.n2 -= 1
                        self.updateServedNbr()
                        self.s = 2
                    else: # no customer waiting
                        deltaTime = self.ServRes
                        self.ArrRes1 -= deltaTime
                        self.ArrRes2 -= deltaTime
                        self.ServRes = 0
                        # State changes
                        self.updateServedNbr()
                        self.s = 0
    
    def simulation(self, N):
        for i in range(N):
            self.singleTransition()
        aveDelay1 = self.totalDelay1/self.nbrServed1
        aveDelay2 = self.totalDelay2/self.nbrServed2
        return (aveDelay1, aveDelay2)                                                           

1,000 Transitions

In [91]:
queueSys = PriorityQueue(lambda1=0.1, lambda2=0.2, t1=0.2, t2=3)
result = queueSys.simulation(1000)
print(">> 1000 simulations <<")
print("Not considering service time:")
print("Average Delay of Level 1 customers:", result[0])
print("Average Delay of Level 2 customers:", result[1])
print("Considering service time:")
print("Average Delay of Level 1 customers:", result[0]+0.2)
print("Average Delay of Level 2 customers:", result[1]+3)

>> 1000 simulations <<
Not considering service time:
Average Delay of Level 1 customers: 1.0490604500384242
Average Delay of Level 2 customers: 2.30442473987603
Considering service time:
Average Delay of Level 1 customers: 1.2490604500384241
Average Delay of Level 2 customers: 5.30442473987603


10,000 Transitions

In [92]:
queueSys = PriorityQueue(lambda1=0.1, lambda2=0.2, t1=0.2, t2=3)
result = queueSys.simulation(10000)
print(">> 10000 simulations <<")
print("Not considering service time:")
print("Average Delay of Level 1 customers:", result[0])
print("Average Delay of Level 2 customers:", result[1])
print("Considering service time:")
print("Average Delay of Level 1 customers:", result[0]+0.2)
print("Average Delay of Level 2 customers:", result[1]+3)

>> 10000 simulations <<
Not considering service time:
Average Delay of Level 1 customers: 0.897428672630251
Average Delay of Level 2 customers: 2.395452173312155
Considering service time:
Average Delay of Level 1 customers: 1.097428672630251
Average Delay of Level 2 customers: 5.3954521733121545


1000 Transitions x 10000

In [93]:
list1 = []
list2 = []
for i in range(10000):
    queueSys = PriorityQueue(lambda1=0.1, lambda2=0.2, t1=0.2, t2=3)
    result = queueSys.simulation(1000)
    list1.append(result[0])
    list2.append(result[1])
print(">> 10000 rounds of simulations <<")
print("Not considering service time:")
print("Average Delay of Level 1 customers:", sum(list1)/10000)
print("Average Delay of Level 2 customers:", sum(list2)/10000)
print("Considering service time:")
print("Average Delay of Level 1 customers:", sum(list1)/10000+0.2)
print("Average Delay of Level 2 customers:", sum(list2)/10000+3)

>> 10000 rounds of simulations <<
Not considering service time:
Average Delay of Level 1 customers: 0.9196298799652395
Average Delay of Level 2 customers: 2.394063942498133
Considering service time:
Average Delay of Level 1 customers: 1.1196298799652395
Average Delay of Level 2 customers: 5.394063942498133
